In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter

In [2]:


# Transform the data to torch.FloatTensor and normalize it
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# Download the training data
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

# Download the test data
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

# Classes in CIFAR-10
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


Files already downloaded and verified
Files already downloaded and verified


In [3]:
class SimpleReluCNN(nn.Module):
    def __init__(self):
        super(SimpleReluCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)  # 3 input channels, 6 output channels, 5x5 kernel
        self.pool = nn.MaxPool2d(2, 2)   # 2x2 max pooling
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)  # 5x5 is the image dimension at this layer
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)  # 10 classes

    def forward(self, x):
        x = self.pool(F.leaky_relu(self.conv1(x)))
        x = self.pool(F.leaky_relu(self.conv2(x)))
        x = torch.flatten(x, 1)  # flatten all dimensions except batch
        x = F.leaky_relu(self.fc1(x))
        x = F.leaky_relu(self.fc2(x))
        x = self.fc3(x)
        return x


class SimpleTanCNN(nn.Module):
    def __init__(self):
        super(SimpleTanCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)  # 3 input channels, 6 output channels, 5x5 kernel
        self.pool = nn.MaxPool2d(2, 2)   # 2x2 max pooling
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)  # 5x5 is the image dimension at this layer
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)  # 10 classes

    def forward(self, x):
        x = self.pool(F.tanh(self.conv1(x)))
        x = self.pool(F.tanh(self.conv2(x)))
        x = torch.flatten(x, 1)  # flatten all dimensions except batch
        x = F.tanh(self.fc1(x))
        x = F.tanh(self.fc2(x))
        x = self.fc3(x)
        return x


In [4]:
def train_and_test_model(model, criterion, optimizer, trainloader, testloader, experiment_name, num_epochs):
    
    # Initialize TensorBoard writer with a descriptive name
    writer = SummaryWriter(f'runs/{experiment_name}')
    for epoch in range(num_epochs):  
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data

            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            # Log the running loss averaged over each 2000 mini-batches
            if i % 2000 == 1999:
                current_loss = running_loss / 2000
                print(f'Epoch {epoch + 1}, Batch {i + 1}, Loss: {current_loss:.3f}, for model: {experiment_name}')
                writer.add_scalar('Training loss', current_loss, epoch * len(trainloader) + i)
                running_loss = 0.0

    print('Finished Training')

    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f'Accuracy of the network on the 10000 test images: {accuracy:.2f}%')

    # Log final accuracy
    writer.add_scalar('Accuracy', accuracy, 0)

    # Close the TensorBoard writer
    writer.close()

    return accuracy

In [5]:

criterion = nn.CrossEntropyLoss()
models = [SimpleReluCNN(),SimpleReluCNN(),SimpleTanCNN(),SimpleTanCNN()]
names = ['relu_and_sgd','relu_and_adam','tanh_and_sgd','tanh_and_adam']

accuracy_dict = {}
for model,name in zip(models,names):
    if 'sgd' in name:
        optimizer = optim.SGD(model.parameters(),lr=1e-4)
    elif 'adam' in name:
        optimizer = optim.Adam(model.parameters(),lr=1e-4)
    accuracy_dict[name] = train_and_test_model(model,criterion,optimizer,trainloader,testloader,name,4)

Epoch 1, Batch 2000, Loss: 2.303, for model: relu_and_sgd
Epoch 1, Batch 4000, Loss: 2.303, for model: relu_and_sgd
Epoch 1, Batch 6000, Loss: 2.302, for model: relu_and_sgd
Epoch 1, Batch 8000, Loss: 2.302, for model: relu_and_sgd
Epoch 1, Batch 10000, Loss: 2.301, for model: relu_and_sgd
Epoch 1, Batch 12000, Loss: 2.302, for model: relu_and_sgd
Epoch 2, Batch 2000, Loss: 2.300, for model: relu_and_sgd
Epoch 2, Batch 4000, Loss: 2.300, for model: relu_and_sgd
Epoch 2, Batch 6000, Loss: 2.300, for model: relu_and_sgd
Epoch 2, Batch 8000, Loss: 2.299, for model: relu_and_sgd
Epoch 2, Batch 10000, Loss: 2.299, for model: relu_and_sgd
Epoch 2, Batch 12000, Loss: 2.298, for model: relu_and_sgd
Epoch 3, Batch 2000, Loss: 2.297, for model: relu_and_sgd
Epoch 3, Batch 4000, Loss: 2.296, for model: relu_and_sgd
Epoch 3, Batch 6000, Loss: 2.296, for model: relu_and_sgd
Epoch 3, Batch 8000, Loss: 2.294, for model: relu_and_sgd
Epoch 3, Batch 10000, Loss: 2.294, for model: relu_and_sgd
Epoch 3, 

In [6]:
accuracy_dict

{'relu_and_sgd': 16.31,
 'relu_and_adam': 54.2,
 'tanh_and_sgd': 21.91,
 'tanh_and_adam': 55.78}